<a href="https://colab.research.google.com/github/tc3oliver/LangChain-Guide/blob/dev/09_load_DynamoDB_local_from_sqlite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import os

drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab Notebooks/LangChain')

Mounted at /content/drive


In [2]:
# 啟動後暫停
!java -Djava.library.path=./dynamodb/DynamoDBLocal_lib -jar ./dynamodb/DynamoDBLocal.jar -sharedDb -dbPath ./dynamodb/db &

KeyboardInterrupt: ignored

In [3]:
!pip install boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 9.8 MB/s eta 0:00:00


In [5]:
import boto3
dynamodb = boto3.resource(
    'dynamodb',
    endpoint_url='http://localhost:8000',
    aws_access_key_id='FAKEFAKEFAKEKEFAKEFA',
    aws_secret_access_key='VNS35gZfPM1pl50y0F3f9LrKrDJ098AtR3T5Jw5A',
    region_name='us-west-2'
)

In [7]:
!pip install langchain==0.0.247 openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.8 MB/s eta 0:00:00


In [16]:
os.environ['AWS_DEFAULT_REGION'] = 'us-west-2'
os.environ['AWS_ACCESS_KEY_ID'] = 'FAKEFAKEFAKEKEFAKEFA'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'VNS35gZfPM1pl50y0F3f9LrKrDJ098AtR3T5Jw5A'

In [17]:
from langchain.memory.chat_message_histories import DynamoDBChatMessageHistory

history = DynamoDBChatMessageHistory(
    table_name="SessionTable",
    endpoint_url='http://localhost:8000',
    session_id='nick')

In [18]:
history.messages

[HumanMessage(content="hi I'm Nick", additional_kwargs={}, example=False),
 AIMessage(content='Hello Nick! How can I assist you today?', additional_kwargs={}, example=False)]

In [20]:
with open('env_vars.txt', 'r') as file:
  for line in file:
    key, value = line.strip().split('=')
    os.environ[key] = value

In [21]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain

def local_bot(session_id):
  llm = ChatOpenAI()
  prompt = ChatPromptTemplate(
      messages=[
          SystemMessagePromptTemplate.from_template(
              "You are a nice chatbot having a conversation with a human."
          ),
          MessagesPlaceholder(variable_name="chat_history"),
          HumanMessagePromptTemplate.from_template("{question}")
      ]
  )
  message_history = DynamoDBChatMessageHistory(table_name="SessionTable", endpoint_url='http://localhost:8000', session_id=session_id)
  memory = ConversationBufferMemory(
      memory_key="chat_history", chat_memory=message_history, return_messages=True
  )
  bot = LLMChain(
      llm=llm,
      prompt=prompt,
      verbose=True,
      memory=memory
  )
  return bot

In [22]:
local_bot("nick").run({"question": "你知道我的名字嗎"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi I'm Nick
AI: Hello Nick! How can I assist you today?
Human: 你知道我的名字嗎

> Finished chain.


'是的，你剛才告訴我你的名字是Nick。有什麼我可以幫助你的嗎？'